In [1]:
import os
import pandas as pd
from matplotlib import pyplot as plt
import cv2
import numpy as np

#data path 수정
os.chdir("C:\\Users\\Owner2\\Desktop\\G팔로미_vuno")
os.getcwd()

'C:\\Users\\Owner2\\Desktop\\G팔로미_vuno'

In [2]:
train_dir = './train_double_input/calc_mass'

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    #featurewise_center=True, #center 중심 유지
    #featurewise_std_normalization=True, #normalization
    rotation_range=25.0,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #brightness_range=[0.2,1.0],
    zoom_range=[0.8,1.2],
    horizontal_flip=True,
    validation_split=0.2) #수평방향 뒤집기

Using TensorFlow backend.


In [4]:
train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir,
        # 모든 이미지의 크기를 150 × 150로 변경합니다
        target_size=(224,224),
        batch_size=32,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(224,224),
    batch_size=32,
    subset='validation') # set as validation data

Found 1370 images belonging to 2 classes.
Found 342 images belonging to 2 classes.


In [5]:
from keras import models, layers
from keras import Input
from keras.applications import VGG16
from keras.models import Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers, initializers, regularizers, metrics
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import BatchNormalization, Conv2D, Activation, Dense, GlobalAveragePooling2D, MaxPooling2D, ZeroPadding2D, Add

model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model.trainable = False
model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [6]:
for layer in model.layers:
    layer.trainable=False

x=model.output
x=layers.Flatten()(x)
outputs=Dense(2, activation='sigmoid')(x)

model1=Model(inputs=model.input, outputs=outputs)

model1.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [7]:
def f1score(precision, recall):
    _f1score = ( 2 * recall * precision) / (recall + precision)
    
    # return a single tensor value
    return _f1score

In [17]:
# checkpoint = ModelCheckpoint(filepath='My_VGG_{epoch:03d}_{val_loss:.7f}.hdf5',monitor='loss', mode='min', save_best_only=True)
checkpoint = ModelCheckpoint(filepath='png_model_type.hdf5', 
            monitor='val_loss', 
            mode='min', 
            save_best_only=True)

In [15]:
import tensorflow as tf

model1.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [16]:
history = model1.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=5,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/5
43/42 [==============================] - 20s 460ms/step - loss: 0.6791 - acc: 0.5686 - auc_1: 0.5410 - precision_1: 0.5189 - recall_1: 0.5189 - f1score: 0.3375 - val_loss: 0.6583 - val_acc: 0.5789 - val_auc_1: 0.6085 - val_precision_1: 0.5718 - val_recall_1: 0.5718 - val_f1score: 0.3415
Epoch 2/5
43/42 [==============================] - 16s 363ms/step - loss: 0.6405 - acc: 0.6511 - auc_1: 0.6327 - precision_1: 0.5884 - recall_1: 0.5884 - f1score: 0.3464 - val_loss: 0.6354 - val_acc: 0.5936 - val_auc_1: 0.6511 - val_precision_1: 0.6053 - val_recall_1: 0.6053 - val_f1score: 0.3466
Epoch 3/5
43/42 [==============================] - 17s 385ms/step - loss: 0.6128 - acc: 0.6920 - auc_1: 0.6665 - precision_1: 0.6179 - recall_1: 0.6179 - f1score: 0.3532 - val_loss: 0.6326 - val_acc: 0.7339 - val_auc_1: 0.6848 - val_precision_1: 0.6337 - val_recall_1: 0.6337 - val_f1score: 0.3525
Epoch 4/5
43/42 [==============================] - 17s 387ms/step - loss: 0.5982 - acc: 0.6869 - auc_1: 0.

In [17]:
for layer in model1.layers[:11]:
    layer.trainable=False

for layer in model1.layers[11:]:
    layer.trainable=True

In [18]:
import tensorflow as tf

model1.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [19]:
history = model1.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=10,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/10
43/42 [==============================] - 17s 405ms/step - loss: 0.4868 - acc: 0.7672 - auc_2: 0.8430 - precision_2: 0.7631 - recall_2: 0.7631 - f1score: 0.3861 - val_loss: 0.4439 - val_acc: 0.7573 - val_auc_2: 0.8514 - val_precision_2: 0.7670 - val_recall_2: 0.7670 - val_f1score: 0.3892
Epoch 2/10
43/42 [==============================] - 16s 381ms/step - loss: 0.3697 - acc: 0.8401 - auc_2: 0.8704 - precision_2: 0.7835 - recall_2: 0.7835 - f1score: 0.4144 - val_loss: 0.2824 - val_acc: 0.8450 - val_auc_2: 0.8877 - val_precision_2: 0.8010 - val_recall_2: 0.8010 - val_f1score: 0.4191
Epoch 3/10
43/42 [==============================] - 16s 381ms/step - loss: 0.3346 - acc: 0.8591 - auc_2: 0.8976 - precision_2: 0.8122 - recall_2: 0.8122 - f1score: 0.4251 - val_loss: 0.3000 - val_acc: 0.8567 - val_auc_2: 0.9051 - val_precision_2: 0.8205 - val_recall_2: 0.8205 - val_f1score: 0.4280
Epoch 4/10
43/42 [==============================] - 16s 378ms/step - loss: 0.3168 - acc: 0.8664 - auc_2

In [20]:
for layer in model1.layers:
    layer.trainable=True

In [21]:
import tensorflow as tf

model1.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-6),
              metrics=['acc',tf.keras.metrics.AUC(name='auc'),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

history = model1.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=37,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/37
43/42 [==============================] - 17s 406ms/step - loss: 0.2252 - acc: 0.9058 - auc: 0.9691 - precision_3: 0.8964 - recall_3: 0.8964 - f1score: 0.4517 - val_loss: 0.1309 - val_acc: 0.8947 - val_auc: 0.9669 - val_precision_3: 0.9034 - val_recall_3: 0.9034 - val_f1score: 0.4449
Epoch 2/37
43/42 [==============================] - 17s 387ms/step - loss: 0.2110 - acc: 0.9182 - auc: 0.9690 - precision_3: 0.9086 - recall_3: 0.9086 - f1score: 0.4545 - val_loss: 0.3379 - val_acc: 0.8918 - val_auc: 0.9694 - val_precision_3: 0.9097 - val_recall_3: 0.9097 - val_f1score: 0.4443
Epoch 3/37
43/42 [==============================] - 17s 390ms/step - loss: 0.2111 - acc: 0.9197 - auc: 0.9696 - precision_3: 0.9106 - recall_3: 0.9106 - f1score: 0.4545 - val_loss: 0.5837 - val_acc: 0.8830 - val_auc: 0.9697 - val_precision_3: 0.9116 - val_recall_3: 0.9116 - val_f1score: 0.4427
Epoch 4/37
43/42 [==============================] - 17s 387ms/step - loss: 0.2091 - acc: 0.9234 - auc: 0.9693 - pre

Epoch 29/37
43/42 [==============================] - 17s 385ms/step - loss: 0.1766 - acc: 0.9365 - auc: 0.9737 - precision_3: 0.9194 - recall_3: 0.9194 - f1score: 0.4624 - val_loss: 0.2449 - val_acc: 0.9064 - val_auc: 0.9737 - val_precision_3: 0.9195 - val_recall_3: 0.9195 - val_f1score: 0.4498
Epoch 30/37
43/42 [==============================] - 17s 385ms/step - loss: 0.1749 - acc: 0.9380 - auc: 0.9738 - precision_3: 0.9196 - recall_3: 0.9196 - f1score: 0.4624 - val_loss: 0.0716 - val_acc: 0.8889 - val_auc: 0.9738 - val_precision_3: 0.9198 - val_recall_3: 0.9198 - val_f1score: 0.4482
Epoch 31/37
43/42 [==============================] - 16s 380ms/step - loss: 0.1706 - acc: 0.9380 - auc: 0.9740 - precision_3: 0.9200 - recall_3: 0.9200 - f1score: 0.4635 - val_loss: 0.4160 - val_acc: 0.9035 - val_auc: 0.9740 - val_precision_3: 0.9202 - val_recall_3: 0.9202 - val_f1score: 0.4468
Epoch 32/37
43/42 [==============================] - 17s 385ms/step - loss: 0.1720 - acc: 0.9321 - auc: 0.9740 -

In [23]:
_loss, _acc, _auc, _precision, _recall, _f1score =  model1.evaluate_generator(validation_generator, steps=32)
print('loss: {:.3f}, accuracy: {:.3f}, auc: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _auc, _precision, _recall, _f1score))

loss: 0.208, accuracy: 0.901, auc: 0.974, precision: 0.921, recall: 0.921, f1score: 0.448


## Make Model2 
output1 : calc[BENIGN, MALIGNANT] <br>
output2 : mass[BENIGN, MALIGNANT]

In [8]:
train_dir2 = './train_double_input/benign_malignant'

In [9]:
train_generator = train_datagen.flow_from_directory(
        # 타깃 디렉터리
        train_dir2,
        # 모든 이미지의 크기를 150 × 150로 변경합니다
        target_size=(224,224),
        batch_size=32,
        subset='training')

validation_generator = train_datagen.flow_from_directory(
    train_dir2, # same directory as training data
    target_size=(224,224),
    batch_size=32,
    subset='validation') # set as validation data

Found 1370 images belonging to 2 classes.
Found 342 images belonging to 2 classes.


In [14]:
for layer in model.layers:
    layer.trainable=False

x=model.output
x=layers.Flatten()(x)
x=Dense(512, activation='relu')(x)
outputs=Dense(2, activation='sigmoid')(x)

model2=Model(inputs=model.input, outputs=outputs)

model2.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [17]:
# checkpoint = ModelCheckpoint(filepath='My_VGG_{epoch:03d}_{val_loss:.7f}.hdf5',monitor='loss', mode='min', save_best_only=True)
checkpoint = ModelCheckpoint(filepath='png_model_pathology_donse2.hdf5', 
            monitor='val_loss', 
            mode='min', 
            save_best_only=True)

In [18]:
import tensorflow as tf

model2.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [19]:
history = model2.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=5,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/5
43/42 [==============================] - 17s 389ms/step - loss: 0.7228 - acc: 0.5208 - auc_2: 0.5118 - precision_2: 0.5069 - recall_2: 0.5626 - f1score: 0.3305 - val_loss: 0.7939 - val_acc: 0.5307 - val_auc_2: 0.5367 - val_precision_2: 0.5212 - val_recall_2: 0.5290 - val_f1score: 0.3360
Epoch 2/5
43/42 [==============================] - 16s 366ms/step - loss: 0.6856 - acc: 0.5420 - auc_2: 0.5463 - precision_2: 0.5256 - recall_2: 0.5344 - f1score: 0.3371 - val_loss: 0.7057 - val_acc: 0.5687 - val_auc_2: 0.5582 - val_precision_2: 0.5330 - val_recall_2: 0.5438 - val_f1score: 0.3470
Epoch 3/5
43/42 [==============================] - 16s 364ms/step - loss: 0.6841 - acc: 0.5588 - auc_2: 0.5633 - precision_2: 0.5389 - recall_2: 0.5495 - f1score: 0.3370 - val_loss: 0.6615 - val_acc: 0.5921 - val_auc_2: 0.5688 - val_precision_2: 0.5440 - val_recall_2: 0.5552 - val_f1score: 0.3482
Epoch 4/5
43/42 [==============================] - 16s 365ms/step - loss: 0.6623 - acc: 0.5872 - auc_2: 0.

In [20]:
for layer in model2.layers[:11]:
    layer.trainable=False

for layer in model2.layers[11:]:
    layer.trainable=True

In [21]:
import tensorflow as tf

model2.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [22]:
history = model2.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=10,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/10
43/42 [==============================] - 17s 389ms/step - loss: 0.6635 - acc: 0.5843 - auc_3: 0.6199 - precision_3: 0.5634 - recall_3: 0.5651 - f1score: 0.3424 - val_loss: 0.6353 - val_acc: 0.5980 - val_auc_3: 0.6325 - val_precision_3: 0.5876 - val_recall_3: 0.5875 - val_f1score: 0.3410
Epoch 2/10
43/42 [==============================] - 16s 362ms/step - loss: 0.6353 - acc: 0.6332 - auc_3: 0.6520 - precision_3: 0.6044 - recall_3: 0.6035 - f1score: 0.3495 - val_loss: 0.6344 - val_acc: 0.6228 - val_auc_3: 0.6570 - val_precision_3: 0.6084 - val_recall_3: 0.6071 - val_f1score: 0.3438
Epoch 3/10
43/42 [==============================] - 16s 373ms/step - loss: 0.6321 - acc: 0.6405 - auc_3: 0.6637 - precision_3: 0.6149 - recall_3: 0.6136 - f1score: 0.3501 - val_loss: 0.7030 - val_acc: 0.6330 - val_auc_3: 0.6694 - val_precision_3: 0.6201 - val_recall_3: 0.6190 - val_f1score: 0.3475
Epoch 4/10
43/42 [==============================] - 16s 363ms/step - loss: 0.6110 - acc: 0.6759 - auc_3

In [23]:
for layer in model2.layers:
    layer.trainable=True

In [24]:
import tensorflow as tf

model2.compile(loss='binary_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-6),
              metrics=['acc',tf.keras.metrics.AUC(name='auc'),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

history = model2.fit_generator(
      train_generator,
      steps_per_epoch = train_generator.samples /32,
      epochs=37,
      validation_data = validation_generator, 
      validation_steps = validation_generator.samples/32,
      callbacks=[checkpoint])

Epoch 1/37
43/42 [==============================] - 17s 388ms/step - loss: 0.5455 - acc: 0.7186 - auc: 0.7863 - precision_4: 0.7102 - recall_4: 0.7124 - f1score: 0.3711 - val_loss: 0.6541 - val_acc: 0.6257 - val_auc: 0.7848 - val_precision_4: 0.7069 - val_recall_4: 0.7132 - val_f1score: 0.3479
Epoch 2/37
43/42 [==============================] - 16s 362ms/step - loss: 0.5498 - acc: 0.7175 - auc: 0.7801 - precision_4: 0.7030 - recall_4: 0.7088 - f1score: 0.3702 - val_loss: 0.6274 - val_acc: 0.6330 - val_auc: 0.7769 - val_precision_4: 0.7028 - val_recall_4: 0.7075 - val_f1score: 0.3507
Epoch 3/37
43/42 [==============================] - 16s 364ms/step - loss: 0.5386 - acc: 0.7401 - auc: 0.7802 - precision_4: 0.7063 - recall_4: 0.7108 - f1score: 0.3730 - val_loss: 0.5925 - val_acc: 0.6301 - val_auc: 0.7800 - val_precision_4: 0.7074 - val_recall_4: 0.7110 - val_f1score: 0.3503
Epoch 4/37
43/42 [==============================] - 16s 361ms/step - loss: 0.5394 - acc: 0.7157 - auc: 0.7806 - pre

Epoch 29/37
43/42 [==============================] - 16s 366ms/step - loss: 0.5090 - acc: 0.7566 - auc: 0.7869 - precision_4: 0.7135 - recall_4: 0.7154 - f1score: 0.3814 - val_loss: 0.6576 - val_acc: 0.6228 - val_auc: 0.7869 - val_precision_4: 0.7137 - val_recall_4: 0.7156 - val_f1score: 0.3463
Epoch 30/37
43/42 [==============================] - 16s 366ms/step - loss: 0.4953 - acc: 0.7635 - auc: 0.7875 - precision_4: 0.7142 - recall_4: 0.7161 - f1score: 0.3849 - val_loss: 0.6833 - val_acc: 0.6345 - val_auc: 0.7877 - val_precision_4: 0.7144 - val_recall_4: 0.7164 - val_f1score: 0.3506
Epoch 31/37
43/42 [==============================] - 16s 367ms/step - loss: 0.5042 - acc: 0.7536 - auc: 0.7881 - precision_4: 0.7148 - recall_4: 0.7167 - f1score: 0.3830 - val_loss: 0.6887 - val_acc: 0.6287 - val_auc: 0.7882 - val_precision_4: 0.7149 - val_recall_4: 0.7169 - val_f1score: 0.3489
Epoch 32/37
43/42 [==============================] - 16s 366ms/step - loss: 0.5010 - acc: 0.7544 - auc: 0.7885 -

In [25]:
_loss, _acc, _auc, _precision, _recall, _f1score =  model2.evaluate_generator(validation_generator, steps=32)
print('loss: {:.3f}, accuracy: {:.3f}, auc: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _auc, _precision, _recall, _f1score))

loss: 0.769, accuracy: 0.601, auc: 0.789, precision: 0.716, recall: 0.717, f1score: 0.346


In [21]:
from keras.models import Model, load_model

new_model = models.load_model('png_VGG16_finetuning.hdf5',compile=False)


new_model.compile(loss='categorical_crossentropy', 
              optimizer=optimizers.Adam(lr=1e-5),
              metrics=['acc',tf.keras.metrics.AUC(),
                      tf.keras.metrics.Precision(),
                      tf.keras.metrics.Recall(),f1score])

In [23]:
test_dir='./test_without_callback'

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(224, 224),
        batch_size=32)

Found 571 images belonging to 4 classes.


In [25]:
_loss, _acc, _auc, _precision, _recall, _f1score =new_model.evaluate_generator(test_generator, steps=test_generator.samples/32)
print('loss: {:.3f}, accuracy: {:.3f}, auc: {:.3f}, precision: {:.3f}, recall: {:.3f}, f1score: {:.3f}'.format(_loss, _acc, _auc, _precision, _recall, _f1score))

loss: 0.901, accuracy: 0.625, auc: 0.868, precision: 0.654, recall: 0.542, f1score: 0.164
